In [1]:
import pandas as pd

In [2]:
# Load data file and take a quick view
df = pd.read_csv("data.csv")
df.head()

,Season,Game,Event_Num,Home_Score,Away_Score
0,2016-17,1,1,0,0
1,2016-17,1,2,0,0
2,2016-17,1,3,0,2
3,2016-17,1,4,0,2
4,2016-17,1,5,0,2


## Part 1: Data Ingestion, Data Manipulation, and Data Export

In [3]:
# Generate Score_Margin variable
df["Score_Margin"] = abs(df["Home_Score"] - df["Away_Score"])

In [4]:
# Save new dataset
df.to_csv("new_data.csv", index=False)

In [5]:
df.head()

,Season,Game,Event_Num,Home_Score,Away_Score,Score_Margin
0,2016-17,1,1,0,0,0
1,2016-17,1,2,0,0,0
2,2016-17,1,3,0,2,2
3,2016-17,1,4,0,2,2
4,2016-17,1,5,0,2,2


## Part 2:  Data Manipulation and Analysis

In [6]:
df = df.sort_values(["Season", "Game", "Event_Num"])

In [7]:
# Get final score margin of each game by getting the value of their last event (numerically sorted).
games_by_season = df.groupby(["Season", "Game"]).Score_Margin.last()

In [8]:
# Generate descriptive statistics of selected season
season = "2020-21"
print(games_by_season[season].describe())

count    1080.000000
mean       11.870370
std         8.940637
min         0.000000
25%         5.000000
50%        10.000000
75%        17.000000
max        57.000000
Name: Score_Margin, dtype: float64


Surprinsingly, the minimum value of the Score_Margin variable is 0 but games cannot finish even. 
Analyzing those games, we find that they include the final score in the Event_Num = 0

In [13]:
# Check Tied Games
games_by_season[games_by_season==0][season]

Game
29     0
90     0
513    0
Name: Score_Margin, dtype: int64

In [11]:
df[(df["Season"]=="2020-21") & (df["Game"]==513)]

,Season,Game,Event_Num,Home_Score,Away_Score,Score_Margin
2290055,2020-21,513,0,109,112,3
2290056,2020-21,513,4,0,0,0
2290057,2020-21,513,7,0,0,0
2290058,2020-21,513,8,0,0,0
2290059,2020-21,513,9,0,0,0
...,...,...,...,...,...,...
2297153,2020-21,513,775,106,111,5
2297154,2020-21,513,776,106,111,5
2297155,2020-21,513,777,106,112,6
2297156,2020-21,513,779,109,112,3


### Option 1

Replace Event_Num=0 for a value higher than any other event number. That way, when sorting events, they will be placed at the bottom. Then, apply the "groupby" described before.

In [16]:
df.Event_Num.replace({0: df.Event_Num.max() + 1}, inplace=True)
df = df.sort_values(["Season", "Game", "Event_Num"])

In [17]:
games_by_season = df.groupby(["Season", "Game"]).Score_Margin.last()

In [18]:
season = "2020-21"

mean = games_by_season[season].mean()
minimum = games_by_season[season].min()
maximum = games_by_season[season].max()
standard_dev = games_by_season[season].std()
variance = standard_dev**2

tableA = pd.DataFrame({"Mean": mean, "Minimum": minimum, "Maximum": maximum, "Standard Deviation": standard_dev, "Variance": variance}, index=[season]).T.round(3)
# tableA.to_excel("TableA.xlsx")
tableA

,2020-21
Mean,12.184
Minimum,1.000
Maximum,57.000
Standard Deviation,9.103
Variance,82.862


### Option 2

For each game, select first the maximum points of the columns "Home_Score" and "Away_Score". Using them, calculate the final score margin. 

In [24]:
alternative_df = df.groupby(["Season", "Game"]).agg({"Home_Score": "max", "Away_Score": "max"})

In [25]:
alternative_df["Score_Margin"] = abs(alternative_df["Home_Score"] - alternative_df["Away_Score"])

In [61]:
season = "2020-21"

points_diff = alternative_df[alternative_df.index.get_level_values(0) == season].Score_Margin

mean = points_diff.mean()
minimum = points_diff.min()
maximum = points_diff.max()
standard_dev = points_diff.std()
variance = standard_dev**2

tableA = pd.DataFrame({"Mean": mean, "Minimum": minimum, "Maximum": maximum, "Standard Deviation": standard_dev, "Variance": variance}, index=[season]).T.round(3)
# tableA.to_excel("TableA.xlsx")
tableA

,2020-21
Mean,12.184
Minimum,1.000
Maximum,57.000
Standard Deviation,9.103
Variance,82.862
